In [168]:
"""
This is a boilerplate pipeline 'data_processing'
generated using Kedro 0.18.3
"""
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

"""
UTILITY FUNCTIONS
"""

'\nUTILITY FUNCTIONS\n'

In [169]:

def lat_lon_bus_stop(bus_stops_df):
    bus_stops_df['lng_lat'] = bus_stops_df['geometry'].str.extract(
        r'\((.*?)\)')
    bus_stops_df[['lon', 'lat']] = bus_stops_df['lng_lat'].str.split(
        " ", 1, expand=True)
    bus_stops_df[['lon', 'lat']] = bus_stops_df[[
        'lon', 'lat']].apply(pd.to_numeric)
    return bus_stops_df[['busstop_id', 'stopplace_type', 'importance_level', 'side_placement', 'geometry', 'lat', 'lon']]


def store_type_lookup(stores_df, plaace_df, store_ids, match):
    combined_df = stores_df.merge(
        plaace_df, how="inner", on="plaace_hierarchy_id")
    pass


"""
TRANSFORMS
"""


def store_type_in_dataaset(stores_df, plaace_df, lv_desc="lv1_desc"):
    combined_df = stores_df.merge(
        plaace_df, how="inner", on="plaace_hierarchy_id")
    return combined_df[lv_desc].value_counts().rename_axis(lv_desc).reset_index(name='count')


def stores_per_location_by_type(stores_df, plaace_df, grunnkrets_df, geo="district_name", lv_desc="lv1_desc"):
    """
    Number of stores of the same type in a geographic location.
    """
    combined_df = stores_df.merge(plaace_df, how="inner", on="plaace_hierarchy_id").merge(
        grunnkrets_df, how="inner", on="grunnkrets_id")
    return combined_df.groupby(by=[geo, lv_desc])['store_id'].count().reset_index(name='count')


def stores_revenue_per_location_by_type(stores_df, plaace_df, grunnkrets_df, geo="district_name", lv_desc="lv1_desc"):
    """
    Total revenue of stores of the same type in a geographic location.
    """
    combined_df = stores_df.merge(plaace_df, how="inner", on="plaace_hierarchy_id").merge(
        grunnkrets_df, how="inner", on="grunnkrets_id")
    return combined_df.groupby(by=[geo, lv_desc])['revenue'].sum().reset_index(name='total_revenue')


def stores_density_per_location_by_type(stores_df, plaace_df, grunnkrets_df, geo="district_name", lv_desc="lv1_desc"):
    """
    Density of stores of the same type in a geographic location.

    This depends on population
    """
    number_of_stores = stores_per_location_by_type(
        stores_df, plaace_df, grunnkrets_df, geo=geo, lv_desc=lv_desc)['count']
    population = 0
    return number_of_stores / population


def stores_in_radius(stores_df, plaace_df, radius=0.1, by_type=False, category=None):
    """
    Number of stores within a given radius. Can also indicate category to filter.
    """
    mat = cdist(stores_df[['lat', 'lon']],
                stores_df[['lat', 'lon']], metric='euclidean')
    new_df = pd.DataFrame(
        mat, index=stores_df['store_id'], columns=stores_df['store_id'])

    if by_type == False:
        count = pd.DataFrame(new_df[(new_df < radius) & (
            new_df > 0)].count(axis=1)).reset_index()
        count.rename(columns={0: 'count'}, inplace=True)
        return count

    else:
        combined_df = stores_df.merge(
            plaace_df, how="inner", on="plaace_hierarchy_id")
        test_df = new_df[(new_df < 0.2) & (new_df > 0)]
        store_count = {}

        for index, row in test_df.iterrows():
            nearby_stores = row.dropna().index.values
            index_type = combined_df[combined_df['store_id']
                                     == index][category].values[0]
            number_same = combined_df[(combined_df['store_id'].isin(nearby_stores)) & (
                combined_df[category] == index_type)]['store_id'].count()
            store_count[index] = number_same

        return pd.DataFrame.from_dict(store_count, orient='index', columns=['count']).reset_index()


def closest_bus_stop_cat(stores_df, bus_stops_df, cat="Regionalt knutepunkt"):
    """
    Id and distance of the closest bus stop to all stores.
    """
    bus_stops_df = bus_stops_df[bus_stops_df['importance_level'] == cat]
    mat = cdist(stores_df[['lat', 'lon']],
                bus_stops_df[['lat', 'lon']], metric='euclidean')

    new_df = pd.DataFrame(
        mat, index=stores_df['store_id'], columns=bus_stops_df['busstop_id'])

    stores = stores_df.store_id
    closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)

    return pd.DataFrame({'store_id': stores.values, 'closest_bus_stop': closest.values, 'distance': distance.values})


def bus_stops_in_radius(stores_df, bus_stops_df, radius=0.1, cat=None):
    """
    Number of bus stops within a given radius. The importance level of bus stops can be specified.
    """
    if cat is not None:
        bus_stops_df = bus_stops_df[bus_stops_df['importance_level'] == cat]

    mat = cdist(stores_df[['lat', 'lon']],
                bus_stops_df[['lat', 'lon']], metric='euclidean')
    new_df = pd.DataFrame(
        mat, index=stores_df['store_id'], columns=bus_stops_df['busstop_id'])
    count = pd.DataFrame(new_df[new_df < radius].count(axis=1)).reset_index()
    count.rename(columns={0: 'count'}, inplace=True)
    return count


# This function calculates the population for each grunnkrets
# Returns a df with grunnkretsID in the first column and population_count in the second column

def population(dataset_age):
    age_df = dataset_age[(dataset_age["year"] == 2016)]
    population = age_df.drop(["grunnkrets_id", "year"], axis=1).sum(axis=1)
    age_df["population_count"] = population
    return age_df[["grunnkrets_id", "population_count"]]

# This function calculates the population in a district or municipality, by setting grouping_elemnt either to the district_name or municipality_name


def population_grouped(data_age, data_geography, grouping_element):
    age_df = population(data_age)
    geography_df = data_geography[data_geography["year"] == 2016]
    population_df = age_df.merge(geography_df, how="left", on="grunnkrets_id")
    grouped_df = population_df.groupby([grouping_element], as_index=False)[
        "population_count"].sum()
    return grouped_df

# This function calculates the density (population/area_km2) for the chosen grouping_element


def population_density(age_df, geo_df, grouping_element):
    age_data = population(age_df)
    geo_df = geo_df[geo_df["year"] == 2016]
    combined_df = age_data.merge(geo_df, how="left", on="grunnkrets_id")
    density_df = combined_df.groupby([grouping_element], as_index=False)[
        ["population_count", "area_km2"]].sum()
    density_df["pop_density"] = density_df["population_count"] / \
        density_df["area_km2"]
    return density_df

# This function checks wether or not a store is part of a mall or not


def is_mall(stores_df):
    df = stores_df.copy()
    df["is_mall"] = df["mall_name"].notna()
    return df[["store_id", "mall_name", "is_mall"]]

# This function checks wether or not a store is part of a chain or not


def is_chain(stores_df):
    df = stores_df.copy()
    df["is_chain"] = df["chain_name"].notna()
    return df[["store_id", "chain_name", "is_chain"]]

# This function calculates the population count per number of stores in a geographic region


def population_per_store(age_df, geo_df, stores_df, grouping_element):
    new_geo_df = geo_df[geo_df["year"] == 2016]
    pop_gk = population(age_df)
    pop_df = population_grouped(age_df, geo_df, grouping_element)
    combined_df = pop_gk.merge(stores_df, how = "left", on = "grunnkrets_id").merge(new_geo_df, how = "left", on = "grunnkrets_id")
    grouped_df = combined_df.groupby([grouping_element], as_index = False)["store_id"].count()
    pop_per_store_df = grouped_df.merge(pop_df, how = "inner", on = grouping_element)
    pop_per_store_df["population_per_num_stores"] = pop_per_store_df["population_count"] / pop_per_store_df["store_id"]
    pop_per_store_df.rename(columns = {"store_id": "num_stores"}, inplace = True)
    return pop_per_store_df


# This function groups the age distrubution (0-90) into 7 buckets with and returns a table which represents the presentages each of these
# buckets corresponds to compared with the total amount of people living in the given geographic region



def age_distrubution(grunnkrets_age_df, geographic_df, grouping_element): 
    age_df = grunnkrets_age_df[grunnkrets_age_df["year"] == 2016]
    age_df1 = age_df.drop(["year"], axis = 1)
    age_df1["num_kids"] = age_df1.iloc[:, 1:8].sum(axis=1)   
    age_df1["num_kids+"] = age_df1.iloc[:, 8:14].sum(axis=1)
    age_df1["num_youths"] = age_df1.iloc[:, 14: 19].sum(axis=1)
    age_df1["num_youthAdult"] = age_df1.iloc[:, 19:27].sum(axis=1)
    age_df1["num_adult"] = age_df1.iloc[:, 27:37].sum(axis=1)
    age_df1["num_adults+"] = age_df1.iloc[:, 37:62].sum(axis=1)
    age_df1["num_pensinors"] = age_df1.iloc[:, 62:92].sum(axis=1)

    age_df2 = age_df1[["grunnkrets_id", "num_kids", "num_kids+", "num_youths", "num_youthAdult", "num_adult", "num_adults+", "num_pensinors"]]

    pop_df = population(grunnkrets_age_df)
    geo_df = geographic_df[geographic_df["year"] == 2016]
    new_geo_df = geo_df.drop(["geometry", "area_km2", "year"], axis = 1)
    combined_df = age_df2.merge(pop_df, how="inner", on = "grunnkrets_id").merge(new_geo_df, how = "inner", on = "grunnkrets_id")
    list_columns = ["num_kids", "num_kids+", "num_youths", "num_youthAdult", "num_adult", "num_adults+", "num_pensinors"]
    combined_df2 = combined_df.groupby([grouping_element], as_index=False)[list_columns].sum()

    pop_gk = population_grouped(grunnkrets_age_df, geographic_df, grouping_element)
    new_df = combined_df2.merge(pop_gk, how = "inner", on = grouping_element)

    new_df["kids_%"] = new_df["num_kids"] / new_df["population_count"]
    new_df["kids+_%"] = new_df["num_kids+"] / new_df["population_count"]
    new_df["youths_%"] = new_df["num_youths"] / new_df["population_count"]
    new_df["youthAdult_%"] = new_df["num_youthAdult"] / new_df["population_count"]
    new_df["adult_%"] = new_df["num_adult"] / new_df["population_count"]
    new_df["adults+_%"] = new_df["num_adults+"] / new_df["population_count"]
    new_df["pensinors_%"] = new_df["num_pensinors"] / new_df["population_count"]

    age_dist_df = new_df.drop(["population_count"], axis = 1)
    #if (grouping_element == "grunnkrets_id"): 
        #return new_df[["grunnkrets_id", "kids_%", "kids+_%", "youths_%", "youthAdult_%", "adult_%", "adults+_%", "pensinors_%" ]]
    #else: 
        #return new_df[[grouping_element, "kids_%", "kids+_%", "youths_%", "youthAdult_%", "adult_%", "adults+_%", "pensinors_%" ]]

    return age_dist_df

# This function calculates the total amount of household types based on a geographic area


def household_type_distrubution(geographic_df, household_df, grouping_element):
    house_df = household_df[household_df["year"] == 2016]
    geo_df = geographic_df[geographic_df["year"] == 2016]
    combined_df = geo_df.merge(house_df, how="inner", on="grunnkrets_id")

    list_columns = ["couple_children_0_to_5_years", "couple_children_18_or_above", "couple_children_6_to_17_years",
                    "couple_without_children", "single_parent_children_0_to_5_years", "single_parent_children_18_or_above",
                    "single_parent_children_6_to_17_years", "singles"]

    grouped_df = combined_df.groupby([grouping_element], as_index=False)[
        list_columns].sum()

    return grouped_df


# Simens functions
def average_revenue_of_chain(dataset_stores):
    "Average revenue of chains in datasett"
    dataset_stores = dataset_stores[(dataset_stores["year"] == 2016)]
    return dataset_stores.groupby(['chain_name'])['revenue'].mean()


def average_revenue_of_mall(dataset_stores):
    "Average revenue of malls in dataset"
    dataset_stores = dataset_stores[(dataset_stores["year"] == 2016)]
    return dataset_stores.groupby(['mall_name'])['revenue'].mean()


def mean_income_per_capita(dataset_age, dataset_income):
    "mean income per capita per grunnkrets"
    age_df = population(dataset_age)
    income_df = dataset_income[dataset_income["year"] == 2016]
    age_and_income_df = age_df.merge(income_df, how='left', on='grunnkrets_id')
    mean_income = age_and_income_df.drop(['year', 'singles', 'couple_without_children',
                                         'couple_with_children', 'other_households', 'single_parent_with_children'], axis=1)
    mean_income['mean_income'] = mean_income['all_households'] / \
        mean_income['population_count']
    mean_income = mean_income.drop(['all_households'], axis=1)

    return mean_income


def mean_income_per_capita_grouped(dataset_age, dataset_income, dataset_geography, grouping_element):
    # gets data from mean_income_per_capita functino
    data_mean_income = mean_income_per_capita(dataset_age, dataset_income)
    # gets data from geography set and makes sure we only use data for 2016
    geography_df = dataset_geography[dataset_geography["year"] == 2016]
    # gets the data of mean income with the geography data
    mean_income_geo_df = data_mean_income.merge(
        geography_df, how='left', on='grunnkrets_id')
    # sum the number of people based on grouping element
    grouped_population_df = mean_income_geo_df.groupby(
        [grouping_element], as_index=False)["population_count"].sum()
    # merge this with the grunnkrets to see both total population per selected area and grunnkrets
    total_grouped_df = mean_income_geo_df.merge(
        grouped_population_df, how='left', on=grouping_element)
    portion_income_df = total_grouped_df
    # find ration of grunnkrets to total population and multiply this with grunnkrets mean income
    portion_income_df['mean_income'] = total_grouped_df['mean_income'] * \
        total_grouped_df['population_count_x'] / \
        total_grouped_df['population_count_y']
    # add these incomes together, should add up to the total mean income for the selected area
    grouped_income_df = portion_income_df.groupby(
        [grouping_element], as_index=False)["mean_income"].sum()
    return grouped_income_df


    

In [170]:
#Reading the datasets 
busstops = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/busstops_norway.csv')
grunnkrets_age = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_age_distribution.csv')
grunnkrets_household = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_income_households.csv')
grunnkrets_norway = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/plaace_hierarchy.csv')
sample_submission = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/sample_submission.csv')
stores_extra = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_extra.csv')
stores_test = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_test.csv')
stores_train = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_train.csv')
stores_with_revenue_data = pd.read_csv("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/notebooks/shared/simens_dataframe.csv")

# Combined dataset based on grunnkrets_id

In [171]:
pop_count = population_grouped(grunnkrets_age, grunnkrets_norway, grouping_element="grunnkrets_id")
pop_count_district = population_grouped(grunnkrets_age, grunnkrets_norway, grouping_element="district_name")
pop_count_municipality = population_grouped(grunnkrets_age, grunnkrets_norway, grouping_element="municipality_name")
pop_count_municipality

<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,municipality_name,population_count
0,Agdenes,1683
1,Alstahaug,7189
2,Alta,19443
3,Alvdal,2202
4,Andebu,5837
...,...,...
417,Østre Toten,14692
418,Øvre Eiker,18002
419,Øyer,4989
420,Øygarden,4800


In [172]:
pop_density = population_density(grunnkrets_age, grunnkrets_norway, grouping_element="grunnkrets_id")
pop_density_district = population_density(grunnkrets_age, grunnkrets_norway, grouping_element="district_name")
pop_density_municipality = population_density(grunnkrets_age, grunnkrets_norway, grouping_element="municipality_name")
pop_density_municipality

<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,municipality_name,population_count,area_km2,pop_density
0,Agdenes,1683,24.198368,69.550145
1,Alstahaug,7189,34.378517,209.113153
2,Alta,19443,44.021857,441.666961
3,Alvdal,2202,30.079660,73.205615
4,Andebu,5837,37.999336,153.607948
...,...,...,...,...
417,Østre Toten,14692,151.675968,96.864389
418,Øvre Eiker,18002,79.545717,226.310112
419,Øyer,4989,38.138484,130.812750
420,Øygarden,4800,10.215312,469.882842


In [173]:
is_mall_df = is_mall(stores_train)
is_mall_df

,store_id,mall_name,is_mall
0,983540538-974187930-44774,Magasinet Drammen,True
1,987074191-973117734-44755,NaN,False
2,984890265-981157303-64491,Kuben Hønefoss,True
3,914057442-992924179-126912,Glasshuspassasjen,True
4,913018583-913063538-668469,Tillertorget,True
...,...,...,...
12854,915789943-915806929-781991,NaN,False
12855,917921733-917982368-868081,CC Gjøvik,True
12856,911721961-911764474-496764,Strømmen Storsenter,True
12857,914337046-914343372-721294,Bystasjonen,True


In [174]:
is_chain_df = is_chain(stores_train)
is_chain_df

,store_id,chain_name,is_chain
0,983540538-974187930-44774,MCDONALDS,True
1,987074191-973117734-44755,MCDONALDS,True
2,984890265-981157303-64491,BURGER KING,True
3,914057442-992924179-126912,BURGER KING,True
4,913018583-913063538-668469,BURGER KING,True
...,...,...,...
12854,915789943-915806929-781991,NaN,False
12855,917921733-917982368-868081,GULATING GRUPPEN,True
12856,911721961-911764474-496764,GULATING GRUPPEN,True
12857,914337046-914343372-721294,NaN,False


In [175]:
pop_per_store_df = population_per_store(grunnkrets_age, grunnkrets_norway, stores_train, grouping_element="grunnkrets_id")
pop_per_store_district_df = population_per_store(grunnkrets_age, grunnkrets_norway, stores_train, grouping_element="district_name")
pop_per_store_municipality_df = population_per_store(grunnkrets_age, grunnkrets_norway, stores_train, grouping_element="municipality_name")
pop_per_store_municipality_df

<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,municipality_name,num_stores,population_count,population_per_num_stores
0,Agdenes,1,1683,1683.000000
1,Alstahaug,22,7189,326.772727
2,Alta,56,19443,347.196429
3,Alvdal,3,2202,734.000000
4,Andebu,6,5837,972.833333
...,...,...,...,...
417,Østre Toten,30,14692,489.733333
418,Øvre Eiker,39,18002,461.589744
419,Øyer,10,4989,498.900000
420,Øygarden,8,4800,600.000000


In [176]:
age_dist_df = age_distrubution(grunnkrets_age, grunnkrets_norway, grouping_element="grunnkrets_id")
age_dist_district_df = age_distrubution(grunnkrets_age, grunnkrets_norway, grouping_element="district_name")
age_dist_municipality_df = age_distrubution(grunnkrets_age, grunnkrets_norway, grouping_element="municipality_name")
age_dist_municipality_df

<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-169-0e6918413b00>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,municipality_name,num_kids,num_kids+,num_youths,num_youthAdult,num_adult,num_adults+,num_pensinors,kids_%,kids+_%,youths_%,youthAdult_%,adult_%,adults+_%,pensinors_%
0,Agdenes,105,103,126,160,153,546,490,0.062389,0.061200,0.074866,0.095068,0.090909,0.324421,0.291147
1,Alstahaug,529,494,472,777,885,2334,1698,0.073585,0.068716,0.065656,0.108082,0.123105,0.324663,0.236194
2,Alta,1798,1596,1494,2407,2780,6029,3339,0.092475,0.082086,0.076840,0.123798,0.142982,0.310086,0.171733
3,Alvdal,196,166,155,217,224,733,511,0.089010,0.075386,0.070391,0.098547,0.101726,0.332879,0.232062
4,Andebu,499,445,384,599,819,1962,1129,0.085489,0.076238,0.065787,0.102621,0.140312,0.336132,0.193421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,Østre Toten,1029,947,910,1410,1583,4953,3860,0.070038,0.064457,0.061938,0.095971,0.107746,0.337122,0.262728
418,Øvre Eiker,1465,1350,1146,1715,2385,6051,3890,0.081380,0.074992,0.063660,0.095267,0.132485,0.336129,0.216087
419,Øyer,319,319,342,552,556,1688,1213,0.063941,0.063941,0.068551,0.110643,0.111445,0.338344,0.243135
420,Øygarden,472,408,332,491,564,1569,964,0.098333,0.085000,0.069167,0.102292,0.117500,0.326875,0.200833


In [177]:
house_type_dist = household_type_distrubution(grunnkrets_norway,grunnkrets_household, grouping_element="grunnkrets_id")
house_type_dist_district = household_type_distrubution(grunnkrets_norway,grunnkrets_household, grouping_element="district_name")
house_type_dist_municipality = household_type_distrubution(grunnkrets_norway,grunnkrets_household, grouping_element="municipality_name")
house_type_dist_municipality

,municipality_name,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles
0,Agdenes,233,181,315,424,34,56,60,322
1,Alstahaug,1180,506,1477,1788,165,175,322,1435
2,Alta,3995,1525,4283,3580,501,673,1209,3511
3,Alvdal,435,205,523,460,53,56,77,452
4,Andebu,1140,410,1273,1410,99,135,252,1000
...,...,...,...,...,...,...,...,...,...
417,Østre Toten,2296,1283,2894,3776,162,445,723,2823
418,Øvre Eiker,3255,1490,3834,4072,270,573,875,3246
419,Øyer,674,472,1050,1270,63,163,220,942
420,Øygarden,1052,347,1132,1052,108,144,162,706


In [178]:
combined1 = pop_count.merge(pop_density, how = "left", on = "grunnkrets_id")
combined2 = combined1.merge(pop_per_store_df, how ="left", on = "grunnkrets_id")
combined3 = combined2.merge(age_dist_df, how ="left", on = "grunnkrets_id")
combined4 = house_type_dist.merge(combined3,  how = "left", on = "grunnkrets_id")
combined4.columns

grunnkrets_id_df = combined4.drop(["population_count_y", "population_count_x", "area_km2"], axis = 1)
grunnkrets_id_df

#grunnkrets_id_df.to_csv("grunnkrets_id_df")

,grunnkrets_id,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles,pop_density,...,num_adult,num_adults+,num_pensinors,kids_%,kids+_%,youths_%,youthAdult_%,adult_%,adults+_%,pensinors_%
0,1010102,53,36,102,180,0,25,26,198,2459.493666,...,56.0,171.0,256.0,0.035032,0.047771,0.054140,0.093949,0.089172,0.272293,0.407643
1,1010103,201,90,215,238,56,47,96,259,2411.878834,...,190.0,373.0,261.0,0.082458,0.075182,0.071140,0.105093,0.153597,0.301536,0.210994
2,1010104,97,42,114,172,15,21,42,149,2076.417612,...,97.0,199.0,201.0,0.077904,0.056657,0.049575,0.111898,0.137394,0.281870,0.284703
3,1010105,262,97,204,566,72,63,116,946,2629.763189,...,310.0,647.0,769.0,0.058773,0.043015,0.036201,0.126917,0.132027,0.275554,0.327513
4,1010107,93,29,100,200,18,14,22,232,852.794956,...,109.0,200.0,178.0,0.058414,0.052851,0.038943,0.172462,0.151599,0.278164,0.247566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13000,20300501,27,7,28,30,0,11,8,30,92.338531,...,17.0,37.0,19.0,0.041667,0.062500,0.104167,0.031250,0.177083,0.385417,0.197917
13001,20300502,20,6,39,34,0,0,4,12,127.260761,...,15.0,41.0,25.0,0.059829,0.076923,0.085470,0.085470,0.128205,0.350427,0.213675
13002,20300503,58,18,35,28,0,7,0,38,140.897931,...,30.0,81.0,42.0,0.116183,0.074689,0.078838,0.095436,0.124481,0.336100,0.174274
13003,20300504,11,21,39,38,0,4,0,36,42.018920,...,15.0,61.0,37.0,0.042945,0.042945,0.079755,0.141104,0.092025,0.374233,0.226994


# Combined dataset based on district_name 

In [179]:
district_df_1 = (pop_count_district
.merge(pop_density_district, how = "left", on = "district_name")
.merge(pop_per_store_district_df, how = "left", on = "district_name")
.merge(age_dist_district_df, how = "left", on = "district_name")
.merge(house_type_dist_district, how = "left", on ="district_name"))

district_df = district_df_1.drop(["population_count_x", "population_count_y", "area_km2"], axis = 1)
district_df

#district_df.to_csv("district_df")



,district_name,pop_density,num_stores,population_count,population_per_num_stores,num_kids,num_kids+,num_youths,num_youthAdult,num_adult,...,adults+_%,pensinors_%,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles
0,Agdenes,69.550145,1,1683,1683.000000,105,103,126,160,153,...,0.324421,0.291147,233,181,315,424,34,56,60,322
1,Alfaset,5952.965088,16,18998,1187.375000,1827,1370,1026,1686,2900,...,0.338983,0.197337,4259,1399,3398,3470,355,500,835,4801
2,Algarheim,289.563424,0,5780,inf,629,564,442,616,922,...,0.338927,0.112111,1368,314,1622,990,127,170,391,825
3,Alstad,3108.781543,5,6400,1280.000000,626,587,537,795,961,...,0.335938,0.116250,1374,443,1853,1160,112,172,411,873
4,Alstahaug/Tjøtta,33.401471,1,817,817.000000,35,54,66,66,77,...,0.341493,0.293758,97,79,234,274,0,18,9,208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,Øymark,115.672993,5,2587,517.400000,165,170,178,243,234,...,0.322768,0.294550,397,170,519,720,14,34,93,537
1478,Øyrekken,1305.617458,0,138,inf,7,6,8,12,7,...,0.333333,0.376812,13,16,26,86,7,6,12,86
1479,Øystese,213.159287,10,2719,271.900000,239,225,166,281,293,...,0.322913,0.234277,561,248,626,564,27,64,71,437
1480,Øystre Slidre nord,80.460331,17,1569,92.294118,95,96,83,139,239,...,0.334608,0.249841,218,130,315,334,13,41,52,362


# Combined dataset based on municipality_name 


In [180]:
municipality_df1 =(pop_count_municipality
.merge(pop_density_municipality, how = "left", on= "municipality_name")
.merge(pop_per_store_municipality_df, how ="left", on = "municipality_name")
.merge(age_dist_municipality_df, how = "left", on ="municipality_name")
.merge(house_type_dist_municipality, how = "left", on = "municipality_name"))

municipality_df = municipality_df1.drop(["population_count_x", "population_count_y", "area_km2"], axis = 1)
municipality_df

#municipality_df.to_csv("municipality_df")


,municipality_name,pop_density,num_stores,population_count,population_per_num_stores,num_kids,num_kids+,num_youths,num_youthAdult,num_adult,...,adults+_%,pensinors_%,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles
0,Agdenes,69.550145,1,1683,1683.000000,105,103,126,160,153,...,0.324421,0.291147,233,181,315,424,34,56,60,322
1,Alstahaug,209.113153,22,7189,326.772727,529,494,472,777,885,...,0.324663,0.236194,1180,506,1477,1788,165,175,322,1435
2,Alta,441.666961,56,19443,347.196429,1798,1596,1494,2407,2780,...,0.310086,0.171733,3995,1525,4283,3580,501,673,1209,3511
3,Alvdal,73.205615,3,2202,734.000000,196,166,155,217,224,...,0.332879,0.232062,435,205,523,460,53,56,77,452
4,Andebu,153.607948,6,5837,972.833333,499,445,384,599,819,...,0.336132,0.193421,1140,410,1273,1410,99,135,252,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,Østre Toten,96.864389,30,14692,489.733333,1029,947,910,1410,1583,...,0.337122,0.262728,2296,1283,2894,3776,162,445,723,2823
418,Øvre Eiker,226.310112,39,18002,461.589744,1465,1350,1146,1715,2385,...,0.336129,0.216087,3255,1490,3834,4072,270,573,875,3246
419,Øyer,130.812750,10,4989,498.900000,319,319,342,552,556,...,0.338344,0.243135,674,472,1050,1270,63,163,220,942
420,Øygarden,469.882842,8,4800,600.000000,472,408,332,491,564,...,0.326875,0.200833,1052,347,1132,1052,108,144,162,706


# Stores_dataset combined with is_mall and is_chain 

In [184]:
stores_df = stores_train[stores_train["year"] == 2016]
is_mall_df = is_mall(stores_df).drop(["mall_name"], axis = 1)
is_chain_df = is_chain(stores_df).drop(["chain_name"], axis = 1)

new_stores_df = stores_df.merge(is_mall_df, how = "left", on = "store_id").merge(is_chain_df, how = "left", on = "store_id")
new_stores_df2 = new_stores_df[["store_id", "grunnkrets_id", "is_mall", "is_chain"]]
new_stores_df2

stores_grunnkrets_merged = new_stores_df2.merge(grunnkrets_id_df, how = "left", on = "grunnkrets_id")
stores_grunnkrets_merged.isna().sum()
stores_grunnkrets_merged.set_index("store_id")


,grunnkrets_id,is_mall,is_chain,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,...,num_adult,num_adults+,num_pensinors,kids_%,kids+_%,youths_%,youthAdult_%,adult_%,adults+_%,pensinors_%
store_id,,,,,,,,,,,,,,,,,,,,,
983540538-974187930-44774,6020303,True,True,20.0,0.0,8.0,28.0,4.0,6.0,11.0,...,32.0,51.0,22.0,0.044586,0.070064,0.031847,0.184713,0.203822,0.324841,0.140127
987074191-973117734-44755,3010306,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
984890265-981157303-64491,6050102,True,True,45.0,9.0,38.0,84.0,13.0,6.0,12.0,...,84.0,85.0,81.0,0.069892,0.067204,0.034946,0.155914,0.225806,0.228495,0.217742
914057442-992924179-126912,18040102,True,True,41.0,10.0,27.0,126.0,11.0,8.0,15.0,...,101.0,173.0,96.0,0.044304,0.033755,0.012658,0.128692,0.213080,0.364979,0.202532
913018583-913063538-668469,16017414,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915789943-915806929-781991,7010705,False,False,159.0,53.0,113.0,488.0,46.0,36.0,58.0,...,210.0,406.0,593.0,0.054090,0.035620,0.031003,0.081794,0.138522,0.267810,0.391161
917921733-917982368-868081,5020406,True,True,47.0,9.0,14.0,138.0,5.0,7.0,19.0,...,116.0,140.0,118.0,0.041750,0.021869,0.011928,0.180915,0.230616,0.278330,0.234592
911721961-911764474-496764,2310803,True,True,147.0,30.0,76.0,332.0,50.0,30.0,33.0,...,217.0,296.0,302.0,0.071620,0.037601,0.013429,0.147717,0.194270,0.264996,0.270367


In [182]:
age_df = grunnkrets_age[grunnkrets_age["year"] == 2016]
stores_df = stores_train[stores_train["year"] == 2016]
household = grunnkrets_household[grunnkrets_household["year"] == 2016]
df = stores_df.merge(age_df, how = "left", on = "grunnkrets_id")
df2 = stores_df.merge(household, how = "left", on = "grunnkrets_id")
df2.isna().sum()

store_id                                    0
year_x                                      0
store_name                                  0
plaace_hierarchy_id                         0
sales_channel_name                          0
grunnkrets_id                               0
address                                  1774
lat                                         0
lon                                         0
chain_name                               9122
mall_name                               10579
revenue                                     0
year_y                                    153
couple_children_0_to_5_years              153
couple_children_18_or_above               153
couple_children_6_to_17_years             153
couple_without_children                   153
single_parent_children_0_to_5_years       153
single_parent_children_18_or_above        153
single_parent_children_6_to_17_years      153
singles                                   153
dtype: int64

In [183]:
simens_dataframe = stores_with_revenue_data
simens_dataframe1 = simens_dataframe.drop(columns = simens_dataframe.columns[0], axis = 1)
simens_dataframe1

,store_id,revenue_chain,revenue_mall,mean_income_grunnkrets_id,mean_income_district_name,mean_income_municipality_name
0,983540538-974187930-44774,23.642250,3.527700,2539.490446,787.771679,988.130324
1,983540538-974187930-44774,23.642250,3.527700,2539.490446,787.771679,988.130324
2,987074191-973117734-44755,23.642250,NaN,NaN,439.648094,358.162526
3,987074191-973117734-44755,23.642250,NaN,NaN,439.648094,358.162526
4,984890265-981157303-64491,7.013545,8.057700,935.215054,609.104173,799.366485
...,...,...,...,...,...,...
25683,911721961-911764474-496764,15.284667,10.366205,394.091316,558.708222,1270.191041
25684,914337046-914343372-721294,NaN,3.042250,1360.854093,1041.140872,699.355134
25685,914337046-914343372-721294,NaN,3.042250,1360.854093,1041.140872,699.355134
25686,917494436-917551936-851089,NaN,NaN,423.565217,789.891892,956.580961
